In [1]:
import joblib
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV
from sklearn.metrics import mean_squared_error, r2_score

from tools.common_methods import get_train_test_data

In [2]:
X_train, X_test, y_train, y_test = get_train_test_data()

In [3]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

# Регуляризація та відсіювання маловпливових ознак
Навчимо модель з L1-регуляризацією, щоб вирішити проблему мультиколінеарності. Коефіцієнти при сильно скорельованих ознаках повинні бути близькі до нуля.

In [4]:
alphas = np.linspace(0.001, 5, 500)
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=30000, n_jobs=-1)
lasso_cv.fit(X_train, y_train)

print('Найменш важливі фічі:', X_train.columns[np.where(lasso_cv.coef_ == 0)])
best = np.where(lasso_cv.coef_ != 0)

Найменш важливі фічі: Index(['Середній пріоритет рекомендованих', 'Рекомендовано за співбесідою',
       'Регіональний коефіцієнт'],
      dtype='object')


In [5]:
lasso = Lasso(alpha=lasso_cv.alpha_, max_iter=30000)
lasso.fit(X_train, y_train)

Lasso(alpha=0.04107214428857715, max_iter=30000)

In [6]:
alphas = np.linspace(0.01, 1000, 2000)
ridge_cv = RidgeCV(alphas=alphas, cv=5)
ridge_cv.fit(X_train[X_train.columns[best]], y_train)
pd.DataFrame(ridge_cv.coef_, X_train.columns[best], columns=["coef"]).sort_values(by="coef", ascending=False)

,coef
Середній пріорітет рекомендованих на загальних підставах,1.151872
квота-1,1.075106
form,0.613948
Макс. обсяг держзамовлення,0.530231
Макс. Бал (на загальних підставах),0.477355
Допущено до конкурсу,0.260255
СЕР,0.200119
Рік,0.178662
uni_code,0.176887
spec_full,0.112935


In [7]:
ridge = Ridge(alpha=ridge_cv.alpha_)
ridge.fit(X_train[X_train.columns[best]], y_train)

Ridge(alpha=262.13844422211105)

In [8]:
y_pred = linreg.predict(X_test)
y_pred_lasso = lasso.predict(X_test)
y_pred_ridge = ridge.predict(X_test[X_train.columns[best]])

print("Linreg:\t MSE =", mean_squared_error(y_test, y_pred), '\t R2 =', r2_score(y_test, y_pred))
print("Lasso: \t MSE =", mean_squared_error(y_test, y_pred_lasso), '\t R2 =', r2_score(y_test, y_pred_lasso))
print("Ridge: \t MSE =", mean_squared_error(y_test, y_pred_ridge), '\t R2 =', r2_score(y_test, y_pred_ridge))

Linreg:	 MSE = 275.79128785392237 	 R2 = 0.8222631534996667
Lasso: 	 MSE = 275.89762664590637 	 R2 = 0.8221946222502028
Ridge: 	 MSE = 275.97643746072777 	 R2 = 0.8221438317201412


In [9]:
joblib.dump(linreg, '../saved_models/linear_regression.sav')
joblib.dump(lasso, '../saved_models/lasso.sav')
joblib.dump(ridge, '../saved_models/ridge.sav')

['../saved_models/ridge.sav']